# Image model training

En este notebook se crearán modelos de reconocimiento de imágenes que distinguen entre la presencia o no de personas y
si estas llevan mascarillas.

Se usará un modelo preentrenado MobileNet V1 con transfer learning.
## Imports

In [1]:
import os

from image_model_training import *

## Parámetros
Se establecen los ajustes para el entrenamiento.

In [2]:
data_origin = EXT # MICRO/EXT
model_type = TINYML # TINYML/COMPLEX

data_type = "image"
project_dir = "/home/daniel/PycharmProjects/tfg-tinyml"

if data_origin == MICRO:
    origin_name = "microcontroller"
    data_dir = f"{project_dir}/samples/microcontroller/color preprocessed image"
    augment_data = True
else:
    origin_name = "external"
    data_dir = f"{project_dir}/samples/external/color preprocessed image"
    augment_data = False

if model_type == TINYML:
    type_name = "tinyml"
    alpha = 0.5
    depth_multiplier = 1.0
else:
    type_name = "complex"
    alpha = 1.0
    depth_multiplier = 1.0

model_name = f"{origin_name} data {type_name} model"

validation_percentage = 20
test_percentage = 10

learning_rate = 0.0001
epochs = 10000

## Carga de datasets
Se obtienen datasets de entrenamiento, validación y test.

In [3]:
training_dataset, validation_dataset, test_dataset = get_image_datasets(data_dir, validation_percentage,
                                                                        test_percentage)


Found 15000 files belonging to 3 classes.
Using 10500 files for training.
Found 15000 files belonging to 3 classes.
Using 4500 files for validation.


## Ajustes de preprocesado
Se aplica o no data augmentation y se aplica un reescalado de valores para que el input sea compatible con MobileNet.

El data augmentation que se puede o no aplicar consiste en rotar horizontalmente la imagen y aplicar una inclinación de
entre 0 y 0.2 radianes.

In [4]:
preprocess_input = tf.keras.applications.mobilenet.preprocess_input

if augment_data:
    data_augmentation = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.15),
    ])
    # data_augmentation_demo(training_dataset, data_augmentation)
else:
    data_augmentation=None

## Construcción de modelo
Se obtiene un modelo usando la parte de MobileNet que extrae las características de la imagen y una capa predictiva que
procesa esas características para realizar predicciones.

In [5]:
model = get_model(alpha, depth_multiplier, preprocess_input, training_dataset, data_augmentation=data_augmentation)
model.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.Accuracy()])
model.summary()

TypeError: Dimension value must be integer or None or have an __index__ method, got value '0.25' with type '<class 'float'>'

## Entrenamiento del modelo
Se entrena el modelo que hemos preparado.

In [ ]:
history = model.fit(training_dataset, epochs=epochs, validation_data=validation_dataset)

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()